In [ ]:
import pandas as pd
from pathlib import Path
import tensorflow as tf
from tensorflow import feature_column as fc
from tensorflow.keras import layers
from tensorflow.keras import models
import matplotlib.pyplot as plt
from dateutil import parser
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping
train_ds = 'D:/PaperWork/personal/AI/VertexAI/training-data-analyst/courses/machine_learning/deepdive2/feature_engineering/labs/../../../data/traffic-taxi-train.csv'
test_ds = 'D:/PaperWork/personal/AI/VertexAI/training-data-analyst/courses/machine_learning/deepdive2/feature_engineering/labs/../../../data/traffic-taxi-test.csv'
validation_ds = 'D:/PaperWork/personal/AI/VertexAI/training-data-analyst/courses/machine_learning/deepdive2/feature_engineering/labs/../../../data/traffic-taxi-valid.csv'

csv_cols = ['fare_amount',
                'pickup_datetime',
                'pickup_longitude',
                'pickup_latitude',
                'dropoff_longitude',
                'dropoff_latitude',
                'passenger_count',
                'trips_last_5min',
                'key']

num_cols = ['pickup_longitude',
            'pickup_latitude',
            'dropoff_longitude',
            'dropoff_latitude',
            'passenger_count']
string_cols = ['pickup_datetime']
label_col = 'fare_amount'
unwanted_cols = ['key','trips_last_5min']
defaults = [[0.0], [''], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0]]
DAYS = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat','Sun']
DAYS_DICT = {idx: day for idx, day in enumerate(DAYS)}

In [ ]:

# print the parent directory of the current file
def read_data(file_path):    
    df = pd.read_csv(file_path, names=csv_cols, header=0, na_values='', dtype={'fare_amount': 'float32',
                                                                           'pickup_datetime': 'string',
                                                                           'pickup_longitude': 'float32',
                                                                           'pickup_latitude': 'float32',
                                                                           'dropoff_longitude': 'float32',
                                                                           'dropoff_latitude': 'float32',
                                                                           'passenger_count': 'float32',
                                                                           'trips_last_5min': 'float32',
                                                                           'key': 'float32'})
    for col in unwanted_cols:
        df.pop(col)
    return df
def convert_pickup_datetime(dt):    
    ts = parser.parse(dt)
    return DAYS_DICT.get(ts.weekday(),'Unknown'), ts.hour

In [ ]:
num_bins=5
df= read_data(train_ds)
longs = np.concatenate([df['pickup_longitude'], df['dropoff_longitude']], axis=0).flatten()
lats = np.concatenate([df['pickup_latitude'], df['dropoff_latitude']], axis=0).flatten()
discretizer_long = layers.Discretization(name='discretize_longitude', num_bins=num_bins)
discretizer_lat = layers.Discretization(name='discretize_latitude', num_bins=num_bins)
discretizer_long.adapt(longs)
discretizer_lat.adapt(lats)

# print(df.head())
# print(df.describe())
def rmse(y_true, y_pred):
    return tf.sqrt(tf.reduce_mean(tf.square(y_true - y_pred)))
def haversine_distance(coords):
    lat1, lon1, lat2, lon2 = coords
    PI = tf.constant(np.pi, dtype=tf.float32)
    R = tf.constant(6371.0, dtype=tf.float32)  # Earth radius in kilometers
    lat1_rad = lat1 * (PI / 180.0)
    lon1_rad = lon1 * (PI / 180.0)
    lat2_rad = lat2 * (PI / 180.0)
    lon2_rad = lon2 * (PI / 180.0)

    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad

    a = tf.math.sin(dlat / 2) ** 2 + tf.math.cos(lat1_rad) * tf.math.cos(lat2_rad) * tf.math.sin(dlon / 2) ** 2
    c = 2 * tf.asin(tf.sqrt(a))

    distance = R * c
    distance.set_shape([None])
    return distance



def build_model(h1_units=128, h2_units=64, dropout_rate=0.3, learning_rate=0.0001):
    
    inputs = {col: layers.Input(name=col, shape=(), dtype='float32') for col in num_cols}
    inputs.update({col: layers.Input(name=col, shape=(), dtype='string') for col in string_cols})
    dt_inp = inputs['pickup_datetime']
    def debug_print(tensor):
        tf.print("\n[DEBUG] Pickup datetime tensor seen by model: ", tensor, summarize=-1)
        return tensor

    #dt_inp = layers.Lambda(debug_print, output_shape=(), name="debug_print")(dt_inp_original)
    # 1. PROCESS NUMERICAL FEATURES
    reshaped_inputs = {col: layers.Reshape(name='reshape_' + col, target_shape=(1,))(inputs[col]) for col in num_cols}
    feature_layer = layers.concatenate(list(reshaped_inputs.values()), name='concatenate_numerical_features')

    # normalize numerical features
    normalizer = layers.Normalization(name='normalizer')
    normalizer.adapt(df[num_cols].to_numpy())
    normalized_feature_layer = normalizer(feature_layer)
    
    # 2. FEATURE ENGINEERING FROM DATETIME STRING
    # prepare a python function to parse a batch of datetime strings into day of week and hour of day
    # to be wrapped in a tf.py_function
    def parse_datetime_batch(dt_tensor):
        dts = [t.decode('utf-8') for t in dt_tensor.numpy()]
        days = []
        hours = []
        for dt in dts:
            day, hour = convert_pickup_datetime(dt)
            days.append(day)
            hours.append(hour)          
        return np.array(days, dtype=np.str_), np.array(hours, dtype=np.float32)
    
    # wrap the python function in a tf.py_function
    def parse_datetime_batch_wrapper(dt_tensor):
        day_of_week_str, hour_of_day_num = tf.py_function(
            func=parse_datetime_batch,
            inp=[dt_tensor],
            Tout=[tf.string, tf.float32]
            )
        day_of_week_str.set_shape([None])
        hour_of_day_num.set_shape([None])
        return day_of_week_str, hour_of_day_num
    day_of_week_str, hour_of_day_num = layers.Lambda(parse_datetime_batch_wrapper, name='parse_datetime')(dt_inp)
    haveersine_dis = layers.Lambda(haversine_distance, output_shape=(), name='haversine_distance')(
        [inputs['pickup_latitude'],
        inputs['pickup_longitude'],
        inputs['dropoff_latitude'],
        inputs['dropoff_longitude']]
    )
    reshape_haveersine_dis = layers.Reshape(name='reshape_haversine_distance', target_shape=(1,))(haveersine_dis)
    # create a lookup layer for day of week
    day_lookup_func = layers.StringLookup(name='day_lookup', vocabulary=DAYS)
    day_lookup = day_lookup_func(day_of_week_str)
    # make it one hot encoded
    day_one_hot_func = layers.CategoryEncoding(name='day_one_hot', num_tokens=len(DAYS)+1, output_mode = 'one_hot')
    day_one_hot = day_one_hot_func(day_lookup)
    
    # reshape the hour_of_day_num to be compatible for concatenation
    reshape_hour_of_day = layers.Reshape(name='reshape_hour_of_day', target_shape=(1,))(hour_of_day_num)
    # normalize hour of day
    hour_normalizer = layers.Normalization(name='hour_normalizer')
    hour_normalizer.adapt(np.arange(0,24).reshape(-1,1))
    normalized_hour_of_day = hour_normalizer(reshape_hour_of_day)

    pickup_latitude_discretized = discretizer_lat(inputs['pickup_latitude'])
    pickup_longitude_discretized = discretizer_long(inputs['pickup_longitude'])
    dropoff_latitude_discretized = discretizer_lat(inputs['dropoff_latitude'])
    dropoff_longitude_discretized = discretizer_long(inputs['dropoff_longitude'])
    # 3. BUILD INTERACTIONS BETWEEN FEATURES
    pickup_loc = layers.HashedCrossing(name='pickup_latitude_x_longitude', num_bins=num_bins*num_bins)([pickup_latitude_discretized, pickup_longitude_discretized])
    pickup_loc_embedding = layers.Embedding(name='pickup_loc_embedding', input_dim=num_bins*num_bins, output_dim=8)(pickup_loc)
    
    dropoff_loc = layers.HashedCrossing(name='dropoff_latitude_x_longitude', num_bins=num_bins*num_bins)([dropoff_latitude_discretized, dropoff_longitude_discretized])
    dropoff_loc_embedding = layers.Embedding(name='dropoff_loc_embedding', input_dim=num_bins*num_bins, output_dim=8)(dropoff_loc)

    pickup_dropoff_loc = layers.HashedCrossing(name='pickup_x_dropoff', num_bins=(num_bins*num_bins)**2)([pickup_loc, dropoff_loc])
    pickup_dropoff_embedding = layers.Embedding(name='pickup_x_dropoff_embedding', input_dim=(num_bins*num_bins)**2, output_dim=16)(pickup_dropoff_loc)
    # 4. ASSEMBLE FINAL FEATURE VECTOR
    # Concatenate the processed numerical and engineered features
    normalized_feature_layer = layers.concatenate([normalized_feature_layer, day_one_hot, normalized_hour_of_day, reshape_haveersine_dis,pickup_loc_embedding, dropoff_loc_embedding, pickup_dropoff_embedding], name='final_concatenate')
    # 5. BUILD THE DNN MODEL
    input_neurons = normalized_feature_layer.shape[-1]*2

    # get the nearest power of 2 lesser than or equal to input_neurons
    def nearest_power_of_2_lesser(n):
        power = 1
        while power <= n:
            power *= 2
        return power // 2

    # get the nearest power of 2 greater than or equal to input_neurons
    def nearest_power_of_2_greater(n):
        power = 1
        while power < n:
            power *= 2
        return power
    def nearest_power_of_2(n):
        lesser = nearest_power_of_2_lesser(n)
        greater = nearest_power_of_2_greater(n)
        if (n - lesser) < (greater - n):
            return lesser
        else:
            return greater
    h1 = layers.Dense(name='hidden_layer_1', units=h1_units, activation='relu')(normalized_feature_layer)
    d1 = layers.Dropout(dropout_rate, name='dropout_1')(h1)
    h2 = layers.Dense(name='hidden_layer_2', units=h2_units, activation='relu')(d1)
    d2 = layers.Dropout(dropout_rate, name='dropout_2')(h2)
    output = layers.Dense(name='output_layer', units=1, activation='linear')(d2)
    model = models.Model(inputs=inputs, outputs=output)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), loss='mse', metrics=['mse', rmse])
    return model

In [ ]:
def features_and_label(row_data):
    for col in unwanted_cols:
        row_data.pop(col)
    label = row_data.pop('fare_amount')
    return row_data, label
# This function will be applied to each batch as it's generated.
def debug_print(row_data):
    tf.print("\\n--- Data batch seen by model ---")
    for key, value in row_data.items():
        # You can print the key as a string and the tensor value
        tf.print("[DEBUG]", key, ": ", value)
    #print(row_data)
    #features,label = row_data
    # Use tf.print, which works inside the TensorFlow graph.
    #tf.print("pickup_datetime dtype:", tf.strings.as_string(features['pickup_datetime'].dtype))
    # Print the first 3 values of the tensor to see what they look like
    #tf.print("pickup_datetime values:", features['pickup_datetime'][:3])
    return row_data
def fix_datetime_dtype(row_data):
    # This uses tf.strings.as_string, which is robust for converting any type.
    row_data['pickup_datetime'] = tf.strings.as_string(row_data['pickup_datetime'])
    return row_data
def ds_to_tf(file, batch_size=32, mode='train'):
    # 1. Load the entire CSV into a pandas DataFrame using your robust function
    df = read_data(file)
    
    # 2. Separate the label from the features
    label = df.pop('fare_amount')
    
    # 3. Create a tf.data.Dataset directly from the pandas DataFrame and labels
    # from_tensor_slices handles the conversion perfectly, respecting the dtypes.
    # We convert the DataFrame of features into a dictionary of tensors.
    tf_ds = tf.data.Dataset.from_tensor_slices((dict(df), label))
    
    # 4. Apply shuffling and batching based on the mode
    if mode == 'train':
        tf_ds = tf_ds.shuffle(buffer_size=batch_size * 10).repeat()
    
    tf_ds = tf_ds.batch(batch_size)
    tf_ds = tf_ds.prefetch(buffer_size=tf.data.AUTOTUNE)
    
    return tf_ds
#def ds_to_tf(file, batch_size=32, mode='train'):
    ds = tf.data.experimental.make_csv_dataset(file,
                                               batch_size=batch_size,
                                               column_names=csv_cols,
                                               column_defaults=defaults,
                                               label_name=None,
                                               num_epochs=1,
                                               shuffle=False)
    ds = ds.map(fix_datetime_dtype)

    ds = ds.map(features_and_label)

    if mode=='train':
        ds = ds.shuffle(buffer_size=batch_size*10).repeat()
        ds = ds.prefetch(buffer_size=tf.data.AUTOTUNE)
    return ds
    label = df.pop('fare_amount')
    tf_ds = tf.data.Dataset.from_tensor_slices((dict(df), label))
    tf_ds = tf_ds.shuffle(buffer_size=batch_size*10)
    tf_ds = tf_ds.batch(batch_size)
    tf_ds = tf_ds.prefetch(buffer_size=tf.data.AUTOTUNE)
    return tf_ds


In [ ]:
train_data_df = read_data(train_ds)
evaluation_data_df = read_data(validation_ds)

TRAIN_BATCH_SIZE = 32 
VALIDATION_BATCH_SIZE = 1000
NUM_TRAIN_EXAMPLES = len(train_data_df)
NUM_EVALS = NUM_TRAIN_EXAMPLES // (TRAIN_BATCH_SIZE)
NUM_EVAL_EXAMPLES = len(evaluation_data_df)
NUM_VALIDATIONS = NUM_EVAL_EXAMPLES // (VALIDATION_BATCH_SIZE)

In [ ]:
# Create datasets for training and evaluation
trainds = ds_to_tf(train_ds, TRAIN_BATCH_SIZE, 'train')
evalds = ds_to_tf(validation_ds, VALIDATION_BATCH_SIZE, 'eval')

# Define early stopping to prevent long, fruitless training runs
early_stopping = EarlyStopping(
    monitor='val_rmse', 
    patience=3,
    verbose=1,
    mode='min',
    restore_best_weights=True
)

# --- HYPERPARAMETER TUNING: HIDDEN LAYER 1 UNITS ---

# 1. Define the values we want to try
h1_units_options = [8, 16, 32, 64, 128, 256]
histories_h1 = {}

print("--- Starting Tuning for Hidden Layer 1 Units ---")
for units in h1_units_options:
    print(f"\nTraining with h1_units = {units}...")
    
    # 2. Build the model with the current value
    # We use the new parameterized function
    model_tuned = build_model(h1_units=units)
    
    # 3. Train the model
    history = model_tuned.fit(
        trainds,
        epochs=15,  # Train for a few more epochs to see the trend
        validation_data=evalds,
        steps_per_epoch=NUM_EVALS,
        validation_steps=NUM_VALIDATIONS,
        callbacks=[early_stopping],
        verbose=0  # Set to 0 to keep the output clean
    )
    
    # 4. Store the results
    histories_h1[f'units_{units}'] = history
    print(f"Finished training for {units} units. Best val_rmse: {min(history.history['val_rmse']):.4f}")

In [ ]:
# --- VISUALIZATION: HIDDEN LAYER 1 UNITS ---
plt.figure(figsize=(12, 5))

# Plot training curves
plt.subplot(1, 2, 1)
for name, history in histories_h1.items():
    plt.plot(history.history['val_rmse'], label=name)
plt.title('Validation RMSE vs. Epochs for h1_units')
plt.ylabel('Validation RMSE')
plt.xlabel('Epoch')
plt.legend()
plt.grid(True)

# Plot best performance
plt.subplot(1, 2, 2)
best_rmses = [min(h.history['val_rmse']) for h in histories_h1.values()]
unit_labels = [str(u) for u in h1_units_options]
plt.bar(unit_labels, best_rmses)
plt.title('Best Validation RMSE per h1_units')
plt.ylabel('Best Validation RMSE')
plt.xlabel('Number of Units in Hidden Layer 1')
plt.grid(axis='y')

plt.tight_layout()
plt.show()

In [ ]:
# --- HYPERPARAMETER TUNING: DROPOUT RATE ---

# 1. Define the values we want to try
dropout_options = [0.1, 0.2, 0.3, 0.4]
histories_dropout = {}

# Use the best h1_units from the previous step (e.g., 128)
best_h1_units = 16  # Replace with the actual best value found 

print("\n\n--- Starting Tuning for Dropout Rate ---")
for rate in dropout_options:
    print(f"\nTraining with dropout_rate = {rate}...")
    
    # 2. Build the model with the current value
    model_tuned = build_model(h1_units=best_h1_units, dropout_rate=rate)
    
    # 3. Train the model
    history = model_tuned.fit(
        trainds,
        epochs=15,
        validation_data=evalds,
        steps_per_epoch=NUM_EVALS,
        validation_steps=NUM_VALIDATIONS,
        callbacks=[early_stopping],
        verbose=0
    )
    
    # 4. Store the results
    histories_dropout[f'rate_{rate}'] = history
    print(f"Finished training for {rate} rate. Best val_rmse: {min(history.history['val_rmse']):.4f}")


# --- VISUALIZATION: DROPOUT RATE ---
plt.figure(figsize=(12, 5))

# Plot training curves
plt.subplot(1, 2, 1)
for name, history in histories_dropout.items():
    plt.plot(history.history['val_rmse'], label=name)
plt.title('Validation RMSE vs. Epochs for Dropout Rate')
plt.ylabel('Validation RMSE')
plt.xlabel('Epoch')
plt.legend()
plt.grid(True)

# Plot best performance
plt.subplot(1, 2, 2)
best_rmses_dropout = [min(h.history['val_rmse']) for h in histories_dropout.values()]
dropout_labels = [str(r) for r in dropout_options]
plt.bar(dropout_labels, best_rmses_dropout)
plt.title('Best Validation RMSE per Dropout Rate')
plt.ylabel('Best Validation RMSE')
plt.xlabel('Dropout Rate')
plt.grid(axis='y')

plt.tight_layout()
plt.show()

In [ ]:
import keras_tuner as kt

# This function is the corrected version of build_keras_tuner_model
def build_keras_tuner_model(hp):
    # --- Define the Hyperparameter Search Space ---
    h1_units = hp.Int('h1_units', min_value=8, max_value=40, step=8)
    h2_units = hp.Int('h2_units', min_value=4, max_value=20, step=4)
    dropout_rate = hp.Float('dropout_rate', min_value=0.1, max_value=0.5, step=0.1)
    learning_rate = hp.Choice('learning_rate', values=[1e-3, 1e-4, 1e-5])
    
    # --- The rest of your model architecture is the same ---
    inputs = {col: layers.Input(name=col, shape=(), dtype='float32') for col in num_cols}
    inputs.update({col: layers.Input(name=col, shape=(), dtype='string') for col in string_cols})
    dt_inp = inputs['pickup_datetime']
    
    # 1. PROCESS NUMERICAL FEATURES
    reshaped_inputs = {col: layers.Reshape(name='reshape_' + col, target_shape=(1,))(inputs[col]) for col in num_cols}
    feature_layer = layers.concatenate(list(reshaped_inputs.values()), name='concatenate_numerical_features')

    # CORRECTION 1: Use the two-step adapt method here
    normalizer = layers.Normalization(name='normalizer')
    normalizer.adapt(df[num_cols].to_numpy()) # df is the global DataFrame we loaded earlier
    normalized_feature_layer = normalizer(feature_layer)
    
    # 2. FEATURE ENGINEERING
    def parse_datetime_batch(dt_tensor):
        dts = [t.decode('utf-8') for t in dt_tensor.numpy()]
        days, hours = [], []
        for dt in dts:
            day, hour = convert_pickup_datetime(dt)
            days.append(day); hours.append(hour)          
        return np.array(days, dtype=np.str_), np.array(hours, dtype=np.float32)

    def parse_datetime_batch_wrapper(dt_tensor):
        day_of_week_str, hour_of_day_num = tf.py_function(func=parse_datetime_batch, inp=[dt_tensor], Tout=[tf.string, tf.float32])
        day_of_week_str.set_shape([None]); hour_of_day_num.set_shape([None])
        return day_of_week_str, hour_of_day_num
        
    day_of_week_str, hour_of_day_num = layers.Lambda(parse_datetime_batch_wrapper, name='parse_datetime')(dt_inp)
    haveersine_dis = layers.Lambda(haversine_distance, name='haversine_distance')([inputs['pickup_latitude'], inputs['pickup_longitude'], inputs['dropoff_latitude'], inputs['dropoff_longitude']])
    reshape_haveersine_dis = layers.Reshape(name='reshape_haversine_distance', target_shape=(1,))(haveersine_dis)
    
    day_lookup_func = layers.StringLookup(name='day_lookup', vocabulary=DAYS)
    day_lookup = day_lookup_func(day_of_week_str)
    day_one_hot_func = layers.CategoryEncoding(name='day_one_hot', num_tokens=len(DAYS)+1, output_mode = 'one_hot')
    day_one_hot = day_one_hot_func(day_lookup)
    
    reshape_hour_of_day = layers.Reshape(name='reshape_hour_of_day', target_shape=(1,))(hour_of_day_num)

    # CORRECTION 2: Use the two-step adapt method here as well
    hour_normalizer = layers.Normalization(name='hour_normalizer')
    hour_normalizer.adapt(np.arange(0,24).reshape(-1,1))
    normalized_hour_of_day = hour_normalizer(reshape_hour_of_day)

    pickup_latitude_discretized = discretizer_lat(inputs['pickup_latitude'])
    pickup_longitude_discretized = discretizer_long(inputs['pickup_longitude'])
    dropoff_latitude_discretized = discretizer_lat(inputs['dropoff_latitude'])
    dropoff_longitude_discretized = discretizer_long(inputs['dropoff_longitude'])
    
    # 3. INTERACTIONS
    pickup_loc = layers.HashedCrossing(name='pickup_latitude_x_longitude', num_bins=num_bins*num_bins)([pickup_latitude_discretized, pickup_longitude_discretized])
    pickup_loc_embedding = layers.Embedding(name='pickup_loc_embedding', input_dim=num_bins*num_bins, output_dim=8)(pickup_loc)
    dropoff_loc = layers.HashedCrossing(name='dropoff_latitude_x_longitude', num_bins=num_bins*num_bins)([dropoff_latitude_discretized, dropoff_longitude_discretized])
    dropoff_loc_embedding = layers.Embedding(name='dropoff_loc_embedding', input_dim=num_bins*num_bins, output_dim=8)(dropoff_loc)
    pickup_dropoff_loc = layers.HashedCrossing(name='pickup_x_dropoff', num_bins=(num_bins*num_bins)**2)([pickup_loc, dropoff_loc])
    pickup_dropoff_embedding = layers.Embedding(name='pickup_x_dropoff_embedding', input_dim=(num_bins*num_bins)**2, output_dim=16)(pickup_dropoff_loc)
    
    # 4. ASSEMBLE
    final_feature_layer = layers.concatenate([normalized_feature_layer, day_one_hot, normalized_hour_of_day, reshape_haveersine_dis, pickup_loc_embedding, dropoff_loc_embedding, pickup_dropoff_embedding], name='final_concatenate')
    
    # 5. BUILD THE DNN MODEL
    h1 = layers.Dense(name='hidden_layer_1', units=h1_units, activation='relu')(final_feature_layer)
    d1 = layers.Dropout(dropout_rate, name='dropout_1')(h1)
    h2 = layers.Dense(name='hidden_layer_2', units=h2_units, activation='relu')(d1)
    d2 = layers.Dropout(dropout_rate, name='dropout_2')(h2)
    output = layers.Dense(name='output_layer', units=1, activation='linear')(d2)
    
    model = models.Model(inputs=inputs, outputs=output)
    
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss='mse',
        metrics=['mse', rmse]
    )
    
    return model

In [ ]:
# Create the datasets
trainds = ds_to_tf(train_ds, TRAIN_BATCH_SIZE, 'train')
evalds = ds_to_tf(validation_ds, VALIDATION_BATCH_SIZE, 'eval')

# Instantiate the tuner
tuner = kt.RandomSearch(
    hypermodel=build_keras_tuner_model,
    objective=kt.Objective("val_rmse", direction="min"), # We want to minimize validation RMSE
    max_trials=10,  # Total number of different hyperparameter combinations to test
    executions_per_trial=1, # Number of times to train each combination
    overwrite=True,
    directory='kt_tuner_dir',
    project_name='taxi_fare_tuning'
)

# Print a summary of the search space
tuner.search_space_summary()

# Define early stopping callback to prevent overfitting during the search
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

# Start the hyperparameter search
print("\n--- Starting Hyperparameter Search ---")
tuner.search(
    trainds,
    epochs=20,
    validation_data=evalds,
    steps_per_epoch=NUM_EVALS,
    validation_steps=NUM_VALIDATIONS,
    callbacks=[stop_early]
)
print("--- Hyperparameter Search Finished ---")

In [ ]:
# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('h1_units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}. The optimal dropout rate is {best_hps.get('dropout_rate')}.
""")

# You can also see a summary of the top results
tuner.results_summary()

# Build the model with the optimal hyperparameters and train it on the full dataset
best_model = tuner.hypermodel.build(best_hps)
history = best_model.fit(
    trainds, 
    epochs=50, 
    validation_data=evalds,
    steps_per_epoch=NUM_EVALS,
    validation_steps=NUM_VALIDATIONS,
    callbacks=[stop_early]
)

# Find the epoch with the best validation RMSE
val_rmse_per_epoch = history.history['val_rmse']
best_epoch = val_rmse_per_epoch.index(min(val_rmse_per_epoch)) + 1
print(f'Best epoch: {best_epoch}')
print(f'Best validation RMSE: {min(val_rmse_per_epoch):.4f}')

In [ ]:
fig = plt.figure(figsize=(10,6))
nr = 1
nc=2

ax = fig.add_subplot(nr,nc,1)
ax.plot(history.history['mse'], label='mse')
ax.plot(history.history['val_mse'], label='val_mse')
ax.set_ylabel('MSE')
ax.set_xlabel('epoch')
ax.legend()
ax = fig.add_subplot(nr,nc,2)
ax.plot(history.history['rmse'], label='rmse')
ax.plot(history.history['val_rmse'], label='val_rmse')
ax.set_ylabel('RMSE')
ax.set_xlabel('epoch')
ax.legend()

In [ ]:
def inspect_dataset_dtypes(dataset, dataset_name, num_batches_to_check=500):
    """
    Iterates through a dataset to check the dtype of 'pickup_datetime'.
    Reports the first batch where the dtype is not tf.string.
    """
    print(f"\n--- Starting inspection of '{dataset_name}' dataset ---")
    problem_found = False
    
    # We iterate using .as_numpy_iterator() which is easier for inspection
    for batch_num, (features, label) in enumerate(dataset.take(num_batches_to_check).as_numpy_iterator()):
        
        datetime_tensor = features.get('pickup_datetime')
        
        if datetime_tensor is None:
            print(f"ERROR: Batch {batch_num} is missing the 'pickup_datetime' key!")
            problem_found = True
            break
            
        # The core of the check: what is the dtype of the numpy array?
        # A tf.string tensor becomes an array of bytes (dtype='O' for object).
        # A tf.float32 tensor becomes an array of floats.
        if datetime_tensor.dtype != 'object':
            print("\n!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            print(f"  PROBLEM DETECTED in '{dataset_name}' at batch number {batch_num}")
            print(f"  Expected 'pickup_datetime' to be strings, but it is of type: {datetime_tensor.dtype}")
            print(f"  Here are the first 5 problematic values: {datetime_tensor[:5]}")
            print("  This is the source of the 'Cast float to string' error.")
            print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!\n")
            problem_found = True
            break
            
        if (batch_num + 1) % 100 == 0:
            print(f"  ... inspected {batch_num + 1} batches of '{dataset_name}', all are tf.string so far.")
            
    if not problem_found:
        print(f"--- SUCCESS: All {num_batches_to_check} inspected batches in '{dataset_name}' have the correct string dtype for 'pickup_datetime'. ---")
    else:
        print(f"--- Inspection of '{dataset_name}' finished. A data type mismatch was found. ---")


# --- Create temporary, non-repeating datasets for inspection ---

# We only need the map that separates features and labels. No shuffle, no repeat.
print("Creating inspection datasets (this may take a moment)...")
inspect_train_ds = tf.data.experimental.make_csv_dataset(train_ds, batch_size=32, column_names=csv_cols, column_defaults=defaults, num_epochs=1, shuffle=False).map(features_and_label)
inspect_eval_ds = tf.data.experimental.make_csv_dataset(validation_ds, batch_size=32, column_names=csv_cols, column_defaults=defaults, num_epochs=1, shuffle=False).map(features_and_label)

# --- Run the inspection ---
inspect_dataset_dtypes(inspect_train_ds, "training (trainds)")
inspect_dataset_dtypes(inspect_eval_ds, "evaluation (evalds)")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# --- 1. Evaluate baseline performance ---
print("Calculating baseline performance on validation set...")
tf.keras.utils.plot_model(best_model, 'dnn_model.png', show_dtype=True, show_shapes=True, rankdir='LR', show_layer_names=True)
# Use your ds_to_tf function in 'eval' mode
validation_data_tf = ds_to_tf(validation_ds, batch_size=VALIDATION_BATCH_SIZE, mode='eval')
baseline_loss, _, baseline_rmse = best_model.evaluate(validation_data_tf, steps=NUM_VALIDATIONS, verbose=0)
print(f"Baseline Validation RMSE: {baseline_rmse:.4f}\n")
# --- 2. Calculate Permutation Importance for each feature ---
# We need the validation data as a pandas DataFrame to easily shuffle columns
df_valid_full = read_data(validation_ds)
labels_valid = df_valid_full.pop('fare_amount')
features_valid = df_valid_full
all_features = num_cols + string_cols
importance_scores = {}
print("Calculating permutation importance...")
for feature in all_features:
# Create a copy to avoid modifying the original validation data
    df_temp = features_valid.copy()
    # Shuffle the values of the current feature
    # .values ensures we shuffle the array, then we put it back
    shuffled_values = df_temp[feature].values
    np.random.shuffle(shuffled_values)
    df_temp[feature] = shuffled_values
    # Convert the modified pandas DataFrame back to a tf.data.Dataset
    # Note: No shuffle or repeat needed for evaluation
    shuffled_ds = tf.data.Dataset.from_tensor_slices((dict(df_temp), labels_valid))
    shuffled_ds = shuffled_ds.batch(VALIDATION_BATCH_SIZE)

    # Evaluate the model on the shuffled data
    shuffled_loss, _, shuffled_rmse = best_model.evaluate(shuffled_ds, verbose=0)

    # The "importance" is the increase in error
    importance_scores[feature] = shuffled_rmse - baseline_rmse
print(f"  Feature: {feature:<20} | Shuffled RMSE: {shuffled_rmse:.4f} | Importance (RMSE increase): {importance_scores[feature]:.4f}")
# --- 3. Plot the results ---
sorted_features = sorted(importance_scores.items(), key=lambda item: item[1])
feature_names = [item[0] for item in sorted_features]
scores = [item[1] for item in sorted_features]
plt.figure(figsize=(10, 6))
plt.barh(feature_names, scores)
plt.xlabel("Increase in RMSE (Higher is More Important)")
plt.title("Permutation Feature Importance")
plt.tight_layout()
plt.show()